In [4]:
## Basic imports
import sys
import os
import time
import pixiedust

# WARNING: These global declarations cause the parallel implementation to 
# crash when executed on Windows
connections = None
networks = None
flowdepthvel = None

from sys import platform
if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    pass
elif platform == "win32":
    print('The parallel version of compute_nhd_routing.py will not execute as currently')
    print('written due to the lack of a fork() capability in the windows OS.')
    print('For parallel execution, please us a *nix OS.')
    print('\nexiting...')
    sys.exit()
    # Some useful references:
    # https://stackoverflow.com/questions/985281/what-is-the-closest-thing-windows-has-to-fork/985525#985525
    # https://stackoverflow.com/questions/8220108/how-do-i-check-the-operating-system-in-python
    # https://stackoverflow.com/questions/6596617/python-multiprocess-diff-between-windows-and-linux

ENV_IS_CL = False
if ENV_IS_CL: root = '/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/'
elif not ENV_IS_CL: 
    sys.setrecursionlimit(4000)
    #root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    root=os.path.abspath('')
    #print(root)
    root=os.path.dirname(os.path.abspath(''))
    #print(root)    
    root=os.path.dirname(os.path.dirname(os.path.abspath('')))
    #print(root)    
    #root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
    print(root)
    
    sys.path.append(os.path.join(root, r'src', r'python_framework'))
    fortran_source_dir = os.path.join(root, r'src', r'fortran_routing', r'mc_pylink_v00', r'MC_singleRCH_singleTS')
#     print(fortran_source_dir) 
    sys.path.append(fortran_source_dir)
print(sys.path)
from mc_singleCh_SingleTStep import compute_mc_reach_up2down 
# import mc_sc_stime as mc
# import mc_sseg_stime_noIC as mc

###############################################
# Build supernetwork_data, supernetwork_values
###############################################

import nhd_network_utilities as nnu
import nhd_reach_utilities as nru

## Muskingum Cunge
import numpy as np

if 1==1:
#     global connections
#     global networks
#     global flowdepthvel

    verbose = True
    debuglevel = 0
    showtiming = True

    test_folder = os.path.join(root, r'test')
    geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')
    print(geo_input_folder)

    #TODO: Make these commandline args
    """##NHD Subset (Brazos/Lower Colorado)"""
#    supernetwork = 'Brazos_LowerColorado_ge5'  #this
    supernetwork= 'Pocono_TEST1'
    """##NHD CONUS order 5 and greater"""
    # supernetwork = 'CONUS_ge5'
    """These are large -- be careful"""
    # supernetwork = 'Mainstems_CONUS'
    # supernetwork = 'CONUS_FULL_RES_v20'
    # supernetwork = 'CONUS_Named_Streams' #create a subset of the full resolution by reading the GNIS field
    # supernetwork = 'CONUS_Named_combined' #process the Named streams through the Full-Res paths to join the many hanging reaches

    if verbose: print('creating supernetwork connections set')
    if showtiming: start_time = time.time()
    #STEP 1
    supernetwork_data, supernetwork_values = nnu.set_networks(
        supernetwork = supernetwork
        , geo_input_folder = geo_input_folder
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        )
    if verbose: print('supernetwork connections set complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))
        

#############################        
# Build networks
#############################            

if 1==1: 
    #STEP 2
    if showtiming: start_time = time.time()
    if verbose: print('organizing connections into networks and reaches ...')
    networks = nru.compose_reaches(
        supernetwork_values
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        , showtiming = showtiming
        )
    if verbose: print('reach organization complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))
        
        
######################
# Build connections
######################

if 1==1:
#STEP 3
    if showtiming: start_time = time.time()
    executiontype = 'serial' # 'parallel'

    if verbose: print('executing serial computation on ordered reaches ...')
    connections = supernetwork_values[0]
    
############################
# Build ordered_reaches
############################
# Actually, all keys of networks dictionary are terminal_segment, the value of 'terminal_segment:' is 
# the key value itself. Now, network takes networks values corresponding to 'terminal_segment' keys. 
for terminal_segment, network in networks.items():
    
#     print(f"for-loop: terminal_segment-> {terminal_segment}")
#     print(f"for-loop: network-> {network}")
#     print()
#     if verbose: print(f"Executing simulation on network {terminal_segment} beginning with streams of order {network['maximum_reach_seqorder']}")
    ordered_reaches = {}
    reach_flowdepthvel = {}
    last_segment_reach={}
    
    # head_segment = links that are either junctions or headwater_reaches belonged to terminal_segment link
    for head_segment, reach in network['reaches'].items():
#         print(f"    for-loop: head_segment-> {head_segment} ,----, reaches-> {reach}")
#         print()
#         print(f"    reach['seqorder']-> {reach['seqorder']},----, ordered_reaches-> {ordered_reaches}")
#         print()
        if reach['seqorder'] not in ordered_reaches:
            ordered_reaches.update({reach['seqorder']:[]}) #TODO: Should this be a set/dictionary?
#             print(f"        ordered_reaches-> {ordered_reaches}")
#             print()            
        ordered_reaches[reach['seqorder']].append([head_segment, reach])
#         print(f"    ordered_reaches-> {ordered_reaches}")
#         print()  

# DONGHA NEW LINE        
        # store last segment link ID for each reach referenced by head_segment
        seg_list=list(reach['segments_list'])
#         segnb=len(reach['segments'])
        lastseg=seg_list[0]
        last_segment_reach[head_segment]= lastseg
#         print(f"last_segment_reach-> {last_segment_reach}")
#         print(f"--------------------------------")
 

/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing
['/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_routing_v02', '/home/dongha.kim/anaconda3/lib/python37.zip', '/home/dongha.kim/anaconda3/lib/python3.7', '/home/dongha.kim/anaconda3/lib/python3.7/lib-dynload', '', '/home/dongha.kim/.local/lib/python3.7/site-packages', '/home/dongha.kim/anaconda3/lib/python3.7/site-packages', '/home/dongha.kim/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/dongha.kim/.ipython', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_framework', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/fortran_routing/mc_pylink_v00/MC_singleRCH_singleTS', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_framework', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/fortr

In [13]:
   
#%%pixie_debugger 
import mc_srch_stime2 as mc
from time import process_time 

if 1==1:
    print(network['maximum_reach_seqorder'])
    print()
    # Start the stopwatch / counter  
    t1_start = process_time()  

## START: hardwired values for mc modules
    ntim=2;       #the number of time steps necessary for variables passed to mc module to compute successfully
#     nlinks=2;     #the number of links needed to define varialbe qd. ** nlinks is not used in fortran source code.
#     mc.var.uslinkid=1
#     mc.var.linkid=2
#     ncomp0=2; mc.var.ncomp0=ncomp0  #the number of segments of a reach upstream of the current reach
    mxseg= 20 #maximum possible number of segments of any single reach in NHD
# END: hardwired values for mc modules
    #MC model key arrays
    mc.var.qd=np.zeros((ntim,mxseg))  #will store MC output qdc (flow downstream current timestep) 
    mc.var.vela=np.zeros((ntim,mxseg)) 
    mc.var.deptha=np.zeros((ntim,mxseg))
    mc.var.dta=np.zeros((mxseg))
    mc.var.dxa=np.zeros((mxseg))
    mc.var.bwa=np.zeros((mxseg))
    mc.var.twa=np.zeros((mxseg))
    mc.var.twcca=np.zeros((mxseg))
    mc.var.na=np.zeros((mxseg))
    mc.var.ncca=np.zeros((mxseg))
    mc.var.csa=np.zeros((mxseg))
    mc.var.soa=np.zeros((mxseg))
    mc.var.qlat=np.zeros((mxseg))
    
    dt_mc= 300.0  #currently used MC simulation time interval 
    nts=1000
    ## START: improvised lateral flow
    bply=nts/2.0; aply= -0.2
    ## END    

    flowdepthvel = {connection:{'flow':np.zeros(nts)
                                , 'depth':np.zeros(nts)
                                , 'vel':np.zeros(nts)
                                , 'qlat':np.zeros(nts)}
                               for connection in connections}         
    
#     for ts in range (0,nts):
        #print(f'timestep: {ts}\n')
        #Note that not networks but network that takes networks values corresponding to 'terminal_segment' keys.
        #Also, network here takes 'terminal_segment' key only at the lower end of a network.      
    for x in range(network['maximum_reach_seqorder'],-1,-1):     
        for head_segment, reach in ordered_reaches[x]:           
            ## Store last segment numbers for upstream reaches for a current head_segment reach 
            usrch_list=list(reach['upstream_reaches'])
            if reach['upstream_reaches']=={0}:
                usrchnb=0
            else:
                usrchnb= len(reach['upstream_reaches'])
#                 print(f"head_segment-> {head_segment}  uprch number-> {usrchnb}")
                
            ## channel data inputs
            seg_list=list(reach['segments_list'])
            seg_list=seg_list[::-1] #to reversed order
            ncomp= len(reach['segments_list']); mc.var.ncomp=ncomp
           
            for seg in range(0,ncomp):                       
                segID= seg_list[seg]
                mc.var.dta[seg]= dt_mc
                mc.var.dxa[seg]= connections[segID]['data'][supernetwork_data['length_col']]
                mc.var.bwa[seg]= connections[segID]['data'][supernetwork_data['bottomwidth_col']]  
                mc.var.twa[seg]= connections[segID]['data'][supernetwork_data['topwidth_col']]  
                mc.var.twcca[seg]= connections[segID]['data'][supernetwork_data['topwidthCC_col']]  
                mc.var.na[seg]= connections[segID]['data'][supernetwork_data['manningn_col']]
                mc.var.ncca[seg]= connections[segID]['data'][supernetwork_data['manningnCC_col']] 
                mc.var.csa[seg]= connections[segID]['data'][supernetwork_data['ChSlp_col']]
                mc.var.soa[seg]= connections[segID]['data'][supernetwork_data['slope_col']]
                
            for ts in range(0,nts):
                for seg in range(0,ncomp):
                    mc.var.qlat[seg]= aply*(ts+1 - bply)**2.0 - aply*bply**2.0 + (seg+1)*10.0  
                # initial values of MC parameters when time is zero and beyond
                if ts==0:
                    # initialize upstream reach
                    mc.var.qus_prev= 0.0
                    # initialize current reach
                    for ii in range(0,ncomp):
                        mc.var.qd[0,ii]= 0.0
                        mc.var.vela[0,ii]= 0.0
                        mc.var.deptha[0,ii]= -999.0
                else:
                    if reach['upstream_reaches'] != {0}:
                        ## when it is first segment of head_segment's reach, compute previous flow from
                        ## the last segments of upstream reaches by adding the flows if the reaches are 
                        ## more than 1.
                        mc.var.uslinkflag=1
                        mc.var.qus_prev=0.0
                        for usrch in range(0,usrchnb):
                            uslsegID= last_segment_reach[usrch_list[usrch]]              
                            mc.var.qus_prev= mc.var.qus_prev + flowdepthvel[uslsegID]['flow'][ts-1]
                    else:
                        mc.var.uslinkflag=0
                        
                # call Fortran routines
                mc.mc.main()

                #output keeping
                for seg in range(0,ncomp):  
                    segID= seg_list[seg]      
                    flowdepthvel[segID]['flow'][ts]= mc.var.qd[1,seg] 
                    flowdepthvel[segID]['vel'][ts]= mc.var.vela[1,seg]  
                    flowdepthvel[segID]['depth'][ts]= mc.var.deptha[1,seg] 
#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.qlat[seg]} FNL:qdc vel depth {mc.var.qd[1,seg]} {mc.var.vela[1,seg]} {mc.var.deptha[1,seg]}") 
#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.qlat[seg]} FNL:qdc vel depth {flowdepthvel[segID]['flow'][ts]} {flowdepthvel[segID]['vel'][ts]} {flowdepthvel[segID]['depth'][ts]}")                         
                
                # store computed values at time k for computation at k+1
                # 1 indicate the current comp. time step k and 0 will be previous time for next time step k+1
                for seg in range(0,ncomp):   
                    mc.var.qd[0,seg]= mc.var.qd[1,seg] 
                    mc.var.vela[0,seg]= mc.var.vela[1,seg]  
                    mc.var.deptha[0,seg]= mc.var.deptha[1,seg]
                          


    # Stop the stopwatch / counter 
    t1_stop = process_time() 
   
    print("Elapsed time:", t1_stop, t1_start)     
    print("Elapsed time during the whole program in seconds:", t1_stop-t1_start) 
            

4

Elapsed time: 6.237682677 5.79865823
Elapsed time during the whole program in seconds: 0.43902444699999954
